In [22]:
import warnings
import tensorflow as tf
import numpy as np
import pandas as pd
import json

seed = 44

tf.compat.v1.random.set_random_seed(seed)
np.random.seed(seed)

In [25]:
df = pd.read_csv('../data/adult_cleaned.csv')
df

,age,education.num,capital.gain,capital.loss,hours.per.week,income,Federal-gov,Local-gov,Never-worked,Private,...,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Unknown_country,Vietnam,Yugoslavia
0,90,9,0,4356,40,<=50K,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,82,9,0,4356,18,<=50K,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,66,10,0,4356,40,<=50K,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,54,4,0,3900,40,<=50K,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,41,10,0,3900,40,<=50K,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,10,0,0,40,<=50K,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
32557,27,12,0,0,38,<=50K,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
32558,40,9,0,0,40,>50K,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
32559,58,9,0,0,40,<=50K,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [26]:
with open('../data/adult_constraints.json', 'r') as f:
    constraints = json.load(f) 
constraints

{'Federal-gov': ['onehot'],
 'Local-gov': ['onehot'],
 'Never-worked': ['onehot'],
 'Private': ['onehot'],
 'Self-emp-inc': ['onehot'],
 'Self-emp-not-inc': ['onehot'],
 'State-gov': ['onehot'],
 'Unknown_workclass': ['onehot'],
 'Without-pay': ['onehot'],
 'Divorced': ['onehot'],
 'Married-AF-spouse': ['onehot'],
 'Married-civ-spouse': ['onehot'],
 'Married-spouse-absent': ['onehot'],
 'Never-married': ['onehot'],
 'Separated': ['onehot'],
 'Widowed': ['onehot'],
 'Adm-clerical': ['onehot'],
 'Armed-Forces': ['onehot'],
 'Craft-repair': ['onehot'],
 'Exec-managerial': ['onehot'],
 'Farming-fishing': ['onehot'],
 'Handlers-cleaners': ['onehot'],
 'Machine-op-inspct': ['onehot'],
 'Other-service': ['onehot'],
 'Priv-house-serv': ['onehot'],
 'Prof-specialty': ['onehot'],
 'Protective-serv': ['onehot'],
 'Sales': ['onehot'],
 'Tech-support': ['onehot'],
 'Transport-moving': ['onehot'],
 'Unknown_occupation': ['onehot'],
 'Amer-Indian-Eskimo': ['onehot', 'non-actionable'],
 'Asian-Pac-Isl

In [30]:

X = df[set(df.columns) - set(['income'])]
Y = df['income']
Y = pd.get_dummies(Y)
Y

,<=50K,>50K
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
32556,1,0
32557,1,0
32558,0,1
32559,1,0


In [31]:
from tensorflow import keras


model = tf.keras.Sequential()
model.add(tf.keras.layers.Input((85,)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(2, activation='softmax'))


model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [32]:
model.fit(
    X.to_numpy(), 
    Y.to_numpy(),
    epochs=100,
    batch_size=128,
    validation_split=0.3,
    shuffle=True,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10)
    ]
)

Train on 22792 samples, validate on 9769 samples
Epoch 1/100
22792/22792 [==============================] - 1s 24us/sample - loss: 11.6480 - acc: 0.6858 - val_loss: 0.4961 - val_acc: 0.8075
Epoch 2/100
22792/22792 [==============================] - 0s 16us/sample - loss: 2.1298 - acc: 0.7607 - val_loss: 0.4534 - val_acc: 0.8075
Epoch 3/100
22792/22792 [==============================] - 0s 16us/sample - loss: 1.0343 - acc: 0.7729 - val_loss: 0.4229 - val_acc: 0.8075
Epoch 4/100
22792/22792 [==============================] - 0s 16us/sample - loss: 0.7606 - acc: 0.7763 - val_loss: 0.3854 - val_acc: 0.8075
Epoch 5/100
22792/22792 [==============================] - 0s 16us/sample - loss: 0.6381 - acc: 0.7742 - val_loss: 0.3748 - val_acc: 0.8075
Epoch 6/100
22792/22792 [==============================] - 0s 16us/sample - loss: 0.5126 - acc: 0.7741 - val_loss: 0.3638 - val_acc: 0.8075
Epoch 7/100
22792/22792 [==============================] - 0s 16us/sample - loss: 0.4642 - acc: 0.7731 - val_l

In [33]:
model.save('../models/adult_NN.h5', overwrite=True, save_format='h5')

In [34]:
model.load_weights('../models/adult_NN.h5')

In [35]:
X.to_numpy()[0].shape

(85,)

In [36]:
model.predict(X.to_numpy()[0:2], verbose=1)


2/2 [==============================] - 0s 15ms/sample


array([[0.37581608, 0.6241839 ],
       [0.37379995, 0.62620014]], dtype=float32)

In [37]:
model.predict_proba(X.to_numpy()[0:2])

array([[0.37581608, 0.6241839 ],
       [0.37379995, 0.62620014]], dtype=float32)

RandomForest

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

In [39]:
classifier = RandomForestClassifier(random_state=seed, criterion='entropy')
classifier.fit(X_train, y_train)


RandomForestClassifier(criterion='entropy', random_state=44)

In [40]:
classifier.score(X_test, y_test)

0.8485003582761798

In [41]:
from joblib import dump, load

dump(classifier, '../models/adult_RF.joblib')

['../models/adult_RF.joblib']

In [42]:
clf = load('../models/adult_RF.joblib')

In [43]:
print(y_test[0:5])
clf.predict(X_test[0:5])

       <=50K  >50K
13099      1     0
21918      1     0
23550      1     0
6892       0     1
17909      0     1


array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1]], dtype=uint8)